Copyright 2022, 2024, 2025 Scott Jensen, San Jose State University

<a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-sa/4.0/88x31.png" /></a><br /><span xmlns:dct="http://purl.org/dc/terms/" property="dct:title">This notebook</span> by <span xmlns:cc="http://creativecommons.org/ns#" property="cc:attributionName">Scott Jensen, Ph.D.</span> is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/">Creative Commons Attribution-ShareAlike 4.0 International License</a>.

#Overview
This notebook is designed to load data into the user's workspace from a data repository on Kaggle.

The markdown is written with my students as the audience. In this notebook there is limited steps they need to do other than running cells since the aim is to get to having the initial data we will then work with.

In **Step 1** you may want the loading of the Kaggle module to not be verbose. 

If you want to use the Yelp data:
* You can use this notebook "as is". Since the data is in a JSON format, creating DataFrames for two of the files (review and user data), is time-consuming, so for those files there is a companion notebook titled `Building Review and User Tables - Serverless` that builds tables from the JSON data files.  For the user data, it also corrects an error in the raw data and summarizes the `friends` field.
* As currently defined, this notebook will download 4 of the 5 data files in the Yelp dataset. The "checkins" file is skipped. There are limited check-ins at most businesses and that data is already summarized (not by user). For those reasons, I never found it useful, but you could easily add it to the `download_list` in Step 4c if you want to include it.

If you want to load data from a different dataset on Kaggle:
* In **Step 4c**, change the `DATASET_NAME` constant to point to the location on Kaggle that you want to load data from.
* Also, in **Step 4c**, update the `download_list` which is a list of dictionaries, where each dictionary defines one of the data files. Currently that dictionary defines the files in the dataset posted on Kaggle by Yelp in 2022. Each dictionary contains the following key:value pairs:
  * raw_name: The name of the file in the dataset on Kaggle.
  * md5: The md5 sum for the unzipped JSON file. This ensures the file downloaded correctly. If you do not want to check if the files downloaded correctly, (my students have not had an error so far - this check was more relevant when students were having to upload the files), you can comment out the call to the function to check the md5 sum in **Step 4c**. In that case, it does not matter what the value is for this key.  
  * name: The name you want to use in Databricks for the file.  Be sure it has a `.bz2` file extension since this notebook will compress the files using the bzip2 file format which Spark can read.
  * If some of the files you download from Kaggle are large, you will want to create tables. There is a companion notebook titled  `Building Review and User Tables - Serverless` that builds tables from the JSON files. This has a generic format that can be used with any file and allows you to write a function with code to do whatever editing you desire before writing the tables. That function is then passed as a parameter to the method for creating the tables.

# Part 1: Loading The Yelp Data

Although the Yelp dataset is not "Big" by commercial standards, for an academic dataset it's large in that unzipped it's approximately 10GB. Fortunately, Spark can work directly with compressed data in certain formats, and we will be loading zipped data using the bzip2 format (WinZip files will not work - don't try it).

Some of the data files, particularly the reviews and the user data files, are nearly 2GB even when compressed, so loading them from a home Internet connection is not possible for many students (keep in mind that if your ISP is a cable company, data download speeds are usually much faster than data upload speeds, and you would need to do both).  If you are curious about your Internet speed, see the <a href="https://www.att.com/support/speedtest/" target="_blank">AT&T speedtest</a> - you would have roughly 900 Mbps both directions when using a wired Ethernet (not Wi-Fi) connection on campus.

In this notebook you will be loading the data directly to your Databricks account from <a href="https://www.kaggle.com/datasets/yelp-dataset/yelp-dataset" target="_blank">Kaggle's</a> website (you will need a free Kaggle account and in step 3 you will be loading your Kaggle credentials). Yelp has staged the data on Kaggle in addition to putting it on the <a href="https://www.yelp.com/dataset" target="_blank">Yelp website</a>.

We will walk through this notebook in class. Since the review and user files are rather large (almost 2GB each when compressed), you will need to complete **Part 2** of this exercise to create tables for the review and user data files. We will use these tables instead of the JSON data when working with the review and user data in future exercises.

<strong><span style="font-size:1.2em;">NOTE:</span></strong> By downloading the dataset from Kaggle, you are bound by Yelp's `Dataset User Agreement` which you can download from the following dataset page at Kaggle: <a href="https://www.kaggle.com/datasets/yelp-dataset/yelp-dataset" target="_blank">https://www.kaggle.com/datasets/yelp-dataset/yelp-dataset</a>.



### Step 1: Setting  up the Kaggle API
The Python code used to download data using Kaggle's API (application programming interface) is in a separate Python module, so we need to install it on the driver node.

Run the cell below to install the module.  You may notice that the  output says that Python may need to be restarted.  **That's why this cell needs to be the first code cell run** - we are 
going to restart python and all of the settings and values in memory will be gone.

**Be sure to wait for this step to complete before continuing.**  When a code cell is finished, it will say how long it took and that you ran it.

In [0]:
pip install kaggle

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


###Step 2 - Restart the cluster
When the above cell finishes (please wait for it), you may see a highlighted message that the Python kernel for the cluster may need to be restarted.  The following cell does that.

If you wondered, `dbutils` (Databricks utilities) is a module containing a number of useful tools for working with Databricks. We will use a few of them in this notebook.


In [0]:
dbutils.library.restartPython()

### Step 3: Setting your Kaggle credentials

The Yelp dataset we will be using is at the following Kaggle page: <a href="https://www.kaggle.com/datasets/yelp-dataset/yelp-dataset" target="_blank">https://www.kaggle.com/datasets/yelp-dataset/yelp-dataset</a>.  If you have not already created an account, you will need to do so before you can download the data. In the upper right-hand corner of the webpage, there will be a `Sign In` link and a `Register` button. If you don't have an account already, click on the `Register` button and create an account.  If you have an account, click the `Sign In` link. If you are already signed in, or once you sign into an existing account or register an account, you will see an icon with what appears to be the head of a goose - that's the link to your profile on Kaggle - click it (if you hover your mouse over the icon, a pop-up tooltip will display your name).

After you click on your profile, click on the `Account` option (gear icon) from the menu that appears.

On your account page you will see a section titled **API** that contains a button titled `Create New API Token` and a button titled `Expire API token`.  Assuming you have not already created an API token, click on the `Create New API Token` button. Kaggle will generate a very tiny file named `kaggle.json` that contains a json object with your user name and key for the API token.  Depending on your browser settings, you will either be prompted to download the file, or it will automatically be downloaded to the Downloads folder on your laptop.  Make sure you can find this file.

<H4>Step 3a: Saving your Kaggle credentials as Databricks secrets<br/>(Please read & perform these steps - this is not just a cell to run)</H4>
Since you should <strong><em>NEVER</em></strong> have your credentials floating around as readable text in a notebook or program, we will use a feature in Databricks known as a "secret" to store your credentials.  We can later programmatically retrieve our credentials from the secrets.

To store your username and key as secrets, you need to use the command line interface (CLI):
* Click **View** in the menu at the top of the screen
*  Select **Cluster tools**
* Select **Web Terminal**

A **Terminal** will appear at the bottom of the screen. The terminal will have a black background. Please wait a second for the prompt to appear - the prompt ends with your email address and a $.

You need to define a "secrets" scope (which we will name kaggle) and then put two secrets into that scope - your Kaggle `username` and `key` from the `kaggle.json` file that you downloaded:
* Open your `kaggle.json` file with a text editor (such as Notepad on a PC).
* Copy/Paste the following line at the Terminal prompt and press enter. DO **NOT** use the Ctrl-V shortcut to paste - that will not work in the Terminal window.  Instead, right-click in the Terminal window at the prompt and select **Paste** to enter the code you copied.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**databricks secrets create-scope kaggle**

* Wait for the system to process that line and return to a prompt.
* Once you are back at a prompt, run each of the following lines and press Enter. As you run each line, it will prompt you for your secret - first your `username` and then your `key`. Wait for the prompt to appear and then copy and paste these values from your kaggle.json file (just the values inside the quotation marks). As you paste the values in, asterisks are displayed (since they are secrets).

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**databricks secrets put-secret kaggle username**

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**databricks secrets put-secret kaggle key**

When the command for adding the key finishes running and you are back at the prompt, you can close the Terminal window.

Once you have set your Kaggle `username` and `key` as Databricks secrets, they will still be available in future sessions if you need to reimport the data, so you won't need to reset them.

#### Step 3b: Create environment variables for your credentials

This step sets environment variables to your Kaggle username and key that you created as Databricks secrets.

***What's an environment variable?*** <a href="https://www.datacamp.com/tutorial/python-environment-variables" target="_blank">environment variables in python</a>


In [0]:
import os

os.environ["KAGGLE_USERNAME"] = dbutils.secrets.get(scope = "kaggle", key = "username")
os.environ["KAGGLE_KEY"] = dbutils.secrets.get(scope = "kaggle", key = "key") 

### Step 4: Importing the Yelp data

In the following code we are loading the files from the Yelp dataset on Kaggle and putting them in the volume `workspace/default/kaggle` in your account.  

If you want some more details on using the Kaggle API for accessing datasets, see their documentation <strong><a href="https://www.kaggle.com/docs/api" target="_blank">here</a></strong> and <strong><a href="https://github.com/Kaggle/kaggle-api#datasets" target="_blank">here</a></strong>.  Also, since the documentation could be better, you may also want to take a look at <strong><a href="https://www.kaggle.com/code/donkeys/kaggle-python-api/notebook" target="_blank">this Kaggle notebook</a></strong> (not a Databricks notebook) that another Kaggle user created.  It explains the API calls in more detail.



Spark can read data files in specific compressed file formats.  You may be familiar with the `zip` file format often used to compress files on Windows computers (also referred to as WinZip). When we download files using Kaggle's API, it automatically zips the files (so it takes less bandwidth to download), ***but does  not append a .zip file extension***.  Unfortunately, Spark cannot read a zip file, but it can read a bzip2 compressed file, which by convention has a `.bz` file extension.  In the following code, we will:

* Download each of the Yelp files to *ephemeral* (temporary) storage space on the driver node for your cluster on AWS
* Unzip the file
* Calculate an MD5 hash (we check this against the hash previously calculated in order to make sure the file downloaded correctly)
* Compress the file using the bzip2 file format
* Move the bzip2 version of the file into the 'workspace/default/kaggle' volume for your account on Databricks

Keep in mind that when we download the files to temporary space on the driver node, that space is gone once the cluster shuts down, but the volume where we move the files is permanent and will be there again when you start a new cluster.

***Let's Go!*** (<span style="color:red;">Step 4c can take 30 minutes to run - be sure it completed successfully before you continue</span>)

#### Step 4a: Creating a directory in ephemeral storage
When we download data from Kaggle, it comes down as a zip file (WinZip) and we need to uncompress it and recompress it in another format (bzip2) that Spark can work with. We need to do this in python before moving the data to the Databricks Catalog. We can do this in the ephemeral storage space on your cluster, but first we need to create a directory in that temporary space where we can work.

**Magics:** In this step we use `%sh` as the first line of the cell. When the first line of a cell starts with a % symbol, it's referred to as a "magic". In Databricks you can use magics to specify that instead of running the cell as python code, you are using another language. In this step we are saying to run the code as Linux shell commands (if you have a Mac, this is like running in the terminal).  In a later step we will use `%SQL` to run a cell as SQL commands.

In [0]:
%sh
mkdir /tmp/kaggle/

#### Step 4b: Creating the volume if it does not exist

The following cell will create a volume named `kaggle` in the `workspace` catalog in the `default` schema.  Every user has a workspace catalog containing the default schema when their account is created.

Here we are using magics to run this cell as a SQL cell instead of a python cell.

In [0]:
%sql

CREATE VOLUME IF NOT EXISTS workspace.default.kaggle

#### Step 4c: Running the import
The following cell loops through the download list to download each file and process it as described above.

#####Explanation:
* The temporary directory being created is named `DOWNLOAD_DIR` and has the value `/tmp/kaggle` This directory only exists for this session. The path starts with `/` because that's the root in Linux (or on your laptop if you are a Mac user). If you are a PC user, it's similar to `c:\`.
* The `DATASET_NAME` is where the Kaggle dataset you are downloading is located. As noted above, the URL for this dataset is <a href="https://www.kaggle.com/datasets/yelp-dataset/yelp-dataset" target="_blank">https://www.kaggle.com/datasets/yelp-dataset/yelp-dataset</a>. You will notice that the path to the dataset is the part of the path after `https://www.kaggle.com/datasets/`. If you are downloading a different dataset, change this value.
* If you want the files stored somewhere other than `/Volumes/workspace/default/kaggle/` in the Databricks Catalog, update the `VOLUME_PATH`.
* If you are downloading a dataset other than the Yelp dataset, update the `download_list` to include a dictionary for each file you are downloading from the dataset specified by `DATASET_NAME`. See the discussion above on the key:value pairs describing each file.
* If you do not want to compare the MD5 checksum, set `VERIFY_FILES` to `False`.
* This step assumes the Volume specified by `VOLUME_PATH` exists. If in Step 4b you changed the path for the volume, be sure to update `VOLUME_PATH` before running this step.



In [0]:
import subprocess
import hashlib
import zipfile
from kaggle import api

DOWNLOAD_DIR = '/tmp/kaggle/'
DATASET_NAME = 'yelp-dataset/yelp-dataset'
VOLUME_PATH = '/Volumes/workspace/default/kaggle/'
VERIFY_FILES = True

download_list = [{"raw_name":"yelp_academic_dataset_business.json", 
                  "md5":"f992501bd77e167970fe6011b9e81573", "name":"business.bz2"},
                 {"raw_name":"yelp_academic_dataset_review.json", 
                  "md5":"56fb6ccb7c0c343e77708b27c79ce1ef", "name":"review.bz2"},
                 {"raw_name":"yelp_academic_dataset_tip.json", 
                  "md5":"822e6c9e34cc117090a5d20b57354677", "name":"tip.bz2"},
                 {"raw_name":"yelp_academic_dataset_user.json", 
                  "md5":"c3ac4d2e223cff9163485e904fc917a7", "name":"user.bz2"}
                 ]

def download_file(raw_name):
  """
  Download the file from Kaggle, unzip it, and recompress it in bzip2 format.
  The file will be downloaded from the Kaggle dataset specified by DATASET_NAME
  and the unzipped file will be in the temporary directory specified by DOWNLOAD_DIR.
  The unzipped raw file will be retained in the ephemeral file space in case the
  MD5 checksum is going to be calculated.

  Parameters:
  raw_name: The name of the file as it appears in the Kaggle dataset
  md5_sum: The MD5 checksum of the unzipped file

  Returns:
  None
  """
  # Download the file to the kaggle directory under /tmp
  api.dataset_download_file(DATASET_NAME,
                                 raw_name,
                                 DOWNLOAD_DIR,
                                 force=True)
  # The file downloaded is a zip file, even though it has the "raw" 
  # name which indicates it's a JSON file. We need to rename it to 
  # have a .zip extention.
  # This has to be done as a shell command because it's the only way to 
  # read/write files in the ephemeral file space.
  subprocess.run(f'mv {DOWNLOAD_DIR}{raw_name} {DOWNLOAD_DIR}{raw_name}.zip', shell=True)
  
  # Unzip the downloaded file into the same /tmp/kaggle directory
  with zipfile.ZipFile(f'{DOWNLOAD_DIR}{raw_name}.zip', 'r') as zip_ref:
    zip_ref.extractall(DOWNLOAD_DIR)
  return


def check_file(raw_name, md5_sum):
  """
  Check the MD5 checksum of the downloaded file. If the checksum doesn't match
  the one passed as the parameter md5_sum, An Exception is raised.

  Parameters:
  raw_name (str): The name of the unzipped file in the temporary directory specified by DOWNLOAD_DIR
  md5_sum (str): The MD5 checksum of the unzipped file

  Returns:  
  None
  """
  with open(f'{DOWNLOAD_DIR}{raw_name}', 'rb') as data_file:
    recalculated_md5sum = hashlib.md5( data_file.read() ).hexdigest()
    if recalculated_md5sum != md5_sum:
      error_msg = (f"For the file: {raw_name}, the MD5 checksum calculated ({recalculated_md5sum})"
                    "does not match the MD5 checksum specified in download_list "
                    f"({md5_sum}).")
      raise Exception(error_msg)
  return 


def save_file(raw_name, file_name):
  """
  Save the file in the bzip2 format to the catalog location specified by VOLUME_PATH.
  This function assumes that the file is in the temporary directory specified by DOWNLOAD_DIR.
  The caltalog location specified by VOLUME_PATH must already exist.

  Parameters:
  raw_name (str): The name of the unzipped file in the temporary directory specified by DOWNLOAD_DIR
  file_name (str): The name used to store the file in the Databricks catalog location specified by VOLUME_PATH

  Returns:
  None
  """
  # Recompress the file in the bzip2 format
  compress_path = f'{DOWNLOAD_DIR}{file_name}'
  with zipfile.ZipFile(str(compress_path), mode='x', compression=zipfile.ZIP_BZIP2) as tempZip:
    tempZip.write(f'{DOWNLOAD_DIR}{raw_name}', arcname=raw_name)
  
  # Move the file to the Kaggle volume
  # Since the "owner" for purposes of the ephemeral file space is the internal ID
  # temporarily assigned to you for this session, the file owner and mode cannot
  # be preserved when moving the file to the volume in the Catalog.
  # When moving, a message will be printed that setting permissions is not supported.
  subprocess.run(f'cp --no-preserve=mode,ownership {DOWNLOAD_DIR}{file_name} {VOLUME_PATH}{file_name}', shell=True)
  subprocess.run(f'rm {DOWNLOAD_DIR}{file_name}', shell=True)
  return


def download_and_process_files(download_list):
  """
  Download and process the files specified in download_list. The files will be downloaded
  from the Kaggle dataset specified by DATASET_NAME and the file will be compressed in
  the bzip2 format and moved to the catalog location specified by VOLUME_PATH. If an error 
  occurs with any of the files, the function will continue to process the remaining files.
  The number of files processed is printed.

  Parameters: 
  download_list: A list of dictionaries spcecifying the files to be downloaded from the 
                 Kaggle dataset specified by DATASET_NAME.
  Returns:
  None  
  """
  processed = 0
  for file_dict in download_list:
    raw_name = file_dict['raw_name']
    md5_sum = file_dict['md5']
    file_name = file_dict['name']
    try:
      download_file(raw_name)
      # Calculate and compare the md5sum
      if VERIFY_FILES:
        check_file(raw_name, md5_sum)
      print(f'Downloaded and unzipped {raw_name}, starting move to {VOLUME_PATH}')
      # moving the file is a time-consuming process
      save_file(raw_name, file_name)
      processed += 1
    except Exception as err:
      print(f'Unexpected error downloading and processing {raw_name}: {err=}, {type(err)=}')
    finally:
      # Clean up the /tmp/kaggle/ directory:
      # - the downloaded file (now has a .zip extention)
      # - the uncompressed file
      try:
        subprocess.run(f'rm {DOWNLOAD_DIR}{raw_name}.zip', shell=True)
        # if the zip file does not exist, the unzipped file will not exist
        subprocess.run(f'rm {DOWNLOAD_DIR}{raw_name}', shell=True) 
      except:
        pass
  print(f'Done downloading and processing {processed} files')
  return

# Process the Files
download_and_process_files(download_list)


Dataset URL: https://www.kaggle.com/datasets/yelp-dataset/yelp-dataset
Downloaded and unzipped yelp_academic_dataset_business.json, starting move to /Volumes/workspace/default/kaggle/
Dataset URL: https://www.kaggle.com/datasets/yelp-dataset/yelp-dataset
Downloaded and unzipped yelp_academic_dataset_review.json, starting move to /Volumes/workspace/default/kaggle/
Dataset URL: https://www.kaggle.com/datasets/yelp-dataset/yelp-dataset
Downloaded and unzipped yelp_academic_dataset_tip.json, starting move to /Volumes/workspace/default/kaggle/
Dataset URL: https://www.kaggle.com/datasets/yelp-dataset/yelp-dataset
Downloaded and unzipped yelp_academic_dataset_user.json, starting move to /Volumes/workspace/default/kaggle/
Done downloading and processing 4 files


# Step 5: Getting the Category Definitions

Above you loaded the Yelp data as zipped files in the bzip2 format to save space.  In the following cell we will take a different approach since it's a small file.  

In the dataset's business file, most businesses have a `categories` field which is a comma-separated list of the categories in which the business operates.  Some categories are at a high level (such as "Restaurants"), but the categories form a hierarchy with increasing levels of detail, so there are more specific categories too, such as "Dim Sum" which is within the "Chinese" category, which in turn is within the "Restaurants" category.  There are over 1500 categories (and growing).  As part of their "Fusion API", Yelp makes this list available to web developers who are creating apps that use Yelp data (and drive traffic to Yelp).  The page documenting the controlled vocabulary for categories can be found **<a href="https://docs.developer.yelp.com/docs/resources-categories" target="_blank">here</a>**.

On Canvas there's a JSON file defining the hierarchy for this controlled vocabulary.  Download that file.  Although there are a lot of categories, as a JSON file this file is tiny compared to the Yelp data, so we don't need to compress it.

### Load the categories.json file through the GUI interface

To load the categories.json through the GUI:
1. Download the `categories.json` file from Canvas.
1. From the menu bar on the left side of the screen, select **Catalog**.
   1. In the catalog hierarchy, under **My Organization**, select `workspace/default/kaggle`. You should see the four files you just loaded listed there. If not, you need to retrace your steps and debug what went wrong.
   1. Click the button labeled **Upload to this volume**.
   1. A pop-up will appear - drag the `categories.json` file there to upload it.
   1. Click the **Upload** button.

You should now see `categories.json` listed with your other Yelp data files.





# Assignment Deliverable

* Make sure you have run all of the code and the correct five files are in the kaggle volume in your catalog.
* **Run the `Building Review and User Tables` notebook** (it will take a while to run).
* Publish ***that*** notebook as you have done before. It will list all of the files you loaded in this notebook.
* Submit the published URL for that notebook as the deliverable for this assignment.